<a href="https://colab.research.google.com/github/Svetorus/Web-scraping/blob/master/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Основы парсинга и работы с API

In [3]:
import pandas as pd
import requests

In [4]:
# метод get
res = requests.get('https://netology.ru/blog/')
# res
res.status_code

200

In [5]:
# браузер отрисовал бы страницу на основе данного текста
res.text

'<!DOCTYPE html>\n<html lang="ru-RU" prefix="og: https://ogp.me/ns#">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="https://gmpg.org/xfn/11">\n<title>Медиа Нетологии: об образовании в диджитале</title>\n\n<meta name="description" content="Издание об онлайн-образовании и карьере в ИТ. Пишем о маркетинге, управлении проектами, дизайне, веб-разработке, Data Science и образовании." />\n<meta name="robots" content="max-image-preview:large" />\n<link rel="canonical" href="https://netology.ru/blog/" />\n<link rel="next" href="https://netology.ru/blog/page/2" />\n<meta property="og:locale" content="ru_RU" />\n<meta property="og:site_name" content="Нетология: образовательная платформа" />\n<meta property="og:type" content="article" />\n<meta property="og:title" content="Медиа Нетологии об образовании и карьере в диджитале 🎓" />\n<meta property="og:description" content="Полезное о маркетинге, управлении проектами,

In [6]:
# получаем плохой статус
bad_request = requests.get('https://netology.ru/blog/some_page')
bad_request

<Response [404]>

In [7]:
bad_request.text

'<!DOCTYPE html>\n<html lang="ru-RU" prefix="og: https://ogp.me/ns#">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="https://gmpg.org/xfn/11">\n<title>Страница не найдена &#8212; Медиа Нетологии</title>\n\n<meta name="robots" content="noindex" />\n<link rel="canonical" href="https://netology.ru/blog/some_page" />\n\n<link rel="preload" href="https://netology.ru/blog/wp-content/plugins/rate-my-post/public/css/fonts/ratemypost.ttf" type="font/ttf" as="font" crossorigin="anonymous">\n\n<script data-cfasync="false" data-pagespeed-no-defer type="text/javascript">//<![CDATA[\n\tvar gtm4wp_datalayer_name = "dataLayer";\n\tvar dataLayer = dataLayer || [];\n//]]>\n</script>\n<link rel=\'dns-prefetch\' href=\'//netology.ru\' />\n<link rel=\'dns-prefetch\' href=\'//fonts.googleapis.com\' />\n<link rel=\'dns-prefetch\' href=\'//s.w.org\' />\n<link rel="alternate" type="application/rss+xml" title="Медиа Нетологии &raq

In [ ]:
# как разбирать всю эту разметку? Поможет BeautifulSoup.
from bs4 import BeautifulSoup

In [ ]:
# создаем объекта, через методы которого будем искать нужные теги и извлекать их содержимое
soup = BeautifulSoup(res.text)
soup

In [ ]:
# функция finda_all позволяет найти все указанные тег с нужными атрибутами (с вложениями), возвращает список
news = soup.find_all('div', class_='posts__item')
print(len(news))
print(news[0])

In [ ]:
for el in news:
    title = el.find('a', 'posts__link').text
    print(title)
    link = el.find('a', 'posts__link').get('href')
    print(link)
    date = el.find('div', 'posts__date').text
    print(date)
    category = el.find('a', 'tags__item').text
    print(category)
    print()

Напишем функцию на основе всех предыдущих действий, которая будет возвращать датафрейм с датой, заголовком, категорией и ссылкой на полный текст поста

In [ ]:
def get_netology_blog_posts():
    url = 'https://netology.ru/blog/'
    req = requests.get(url).text
    soup = BeautifulSoup(req)

    news = soup.find_all('div', class_='posts__item')

    netology_blog = pd.DataFrame()

    for el in news:
        title = el.find('a', 'posts__link').text
        link = el.find('a', 'posts__link').get('href')
        date = el.find('div', 'posts__date').text
        category = el.find('a', 'tags__item').text
        row = {'date': date, 'title': title, 'link': link, 'category':category}
        netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
    return netology_blog.reset_index(drop=True)

get_netology_blog_posts()

А теперь поработаем поиском по сайту

In [ ]:
# cформируем поисковый запрос, обратите внимание на его формат
URL = 'https://netology.ru/blog/?s=python'

In [ ]:
req = requests.get(URL)
req.text

В функцию get можно передавать параметры и заголовки запроса в виде словаря

In [ ]:
# в request можно передать параметры запроса и заголовки (headers) в виде словарей. 
URL = 'https://netology.ru/blog/'
params = {
    's': 'python'
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Mobile Safari/537.36'
}

req = requests.get(URL, params=params, headers=headers)

req.text

In [ ]:
soup = BeautifulSoup(req.text)
articles = soup.find_all('article', class_='status-publish')
len(articles)

In [ ]:
netology_blog = pd.DataFrame()
for article in articles:
    title = article.find('h2', class_='entry-title').text
    link = article.find('h2', class_='entry-title').find('a').get('href')
    date = article.find('span', class_='posted-on').text.strip()
    category = article.find('div', class_='entry-cats').text
    row = {'date': date, 'title': title, 'link': link, 'category':category}
    netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
netology_blog

In [ ]:
netology_blog = pd.DataFrame()
for article in articles:
    title = article.find('h2', class_='entry-title').text
    link = article.find('h2', class_='entry-title').find('a').get('href')
    date = article.find('span', class_='posted-on').text.strip()
    if article.find('div', class_='entry-cats'):
        category = article.find('div', class_='entry-cats').text
    else:
        category = 'Новость'
    row = {'date': date, 'title': title, 'link': link, 'category':category}
    netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
netology_blog

А что если хотим собрать не только начальную выдачу, но и другие страницы? Изучам запросы при соответствующих действиях на сайте

In [ ]:
# адрес для отправки запроса
url = 'https://netology.ru/blog/?infinity=scrolling'

In [ ]:
# параметры для отправки запроса
params = {
    'page': 1,
    'query_args[s]': 'python'
}

In [ ]:
# здесь мы используем post запрос
res = requests.post(url, params=params)
res.text

In [ ]:
soup = BeautifulSoup(res.json()['html']) # извлекаем из ответа html-разметку
display(soup)  

In [ ]:
articles = soup.find_all('article', class_='status-publish')

netology_blog = pd.DataFrame()
for article in articles:
    title = article.find('h2', class_='entry-title').text
    link = article.find('h2', class_='entry-title').find('a').get('href')
    date = article.find('span', class_='posted-on').text.strip()
    if article.find('div', class_='entry-cats'):
        category = article.find('div', class_='entry-cats').text
    else:
        category = 'Новость'    
    row = {'date': date, 'title': title, 'link': link, 'category':category}
    netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
netology_blog

In [ ]:
# напишем функцию по предыдущему коду
def get_posts_from_blog():
    url = 'https://netology.ru/blog/?infinity=scrolling'    
    params = {
        'page': 1,
        'query_args[s]': 'python'
    }
    res = requests.post(url, params=params)

    soup = BeautifulSoup(res.json()['html']) 
    articles = soup.find_all('article', class_='status-publish')

    netology_blog = pd.DataFrame()
    for article in articles:
        title = article.find('h2', class_='entry-title').text
        link = article.find('h2', class_='entry-title').find('a').get('href')
        date = article.find('span', class_='posted-on').text.strip()
        if article.find('div', class_='entry-cats'):
            category = article.find('div', class_='entry-cats').text
        else:
            category = 'Новость'        
        row = {'date': date, 'title': title, 'link': link, 'category':category}
        netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
    return netology_blog.reset_index(drop=True)

get_posts_from_blog()

А как извлечь посты с нескольких страниц?

In [ ]:
# импортируем библиотеку для задержки исполнения кода
import time

def get_posts_from_blog(pages):
    url = 'https://netology.ru/blog/?infinity=scrolling'    
    params = {
        'query_args[s]': 'python'
    }

    netology_blog = pd.DataFrame() 
    # добавим цикл с перебором страниц
    for page in range(1, pages+1):
        params['page'] = page
        res = requests.post(url, params=params)
        time.sleep(0.33)
        soup = BeautifulSoup(res.json()['html']) 
        articles = soup.find_all('article', class_='status-publish')

        for article in articles:
            title = article.find('h2', class_='entry-title').text
            link = article.find('h2', class_='entry-title').find('a').get('href')
            date = article.find('span', class_='posted-on').text.strip()
            if article.find('div', class_='entry-cats'):
                category = article.find('div', class_='entry-cats').text
            else:
                category = 'Новость'     
            row = {'date': date, 'title': title, 'link': link, 'category':category}
            netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 

    return netology_blog.reset_index(drop=True)

get_posts_from_blog(5)

А если хотим добавить в датафрейм полные тексты всех постов?

In [ ]:
blog_posts = get_posts_from_blog(5)

def add_full_text(posts_df):
    i = 0
    for el in posts_df['link']:
        # print(el)
        req = requests.get(el).text
        time.sleep(0.33)
        soup = BeautifulSoup(req)
        full_text = soup.find('div', class_='entry-content').text.strip()
        posts_df.loc[i, 'text'] = full_text
        i += 1
    return posts_df

add_full_text(blog_posts)

Напишем скрипт, который будет собирать новости с сайта Коммерсанта

In [9]:
URL = 'https://www.kommersant.ru/search/results'
params = {
    'search_query': 'python'
}

In [10]:
res = requests.get(URL, params)

In [11]:
res.text

'<!DOCTYPE html>\r\n<html class="no-js" lang="ru">\r\n<head>\r\n    <title>Коммерсантъ: последние новости России и мира</title>\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge, chrome=1" />\r\n    <meta charset="utf-8" />\r\n\r\n    <meta name="format-detection" content="telephone=no" />\r\n    <meta name="title" content="Коммерсантъ: последние новости России и мира" />\r\n    <meta name="description" content="Актуальные новости, объективный анализ и эксклюзивные комментарии о важнейших событиях и трендах" />\r\n    <meta name="keywords" content="Новости,Политика,Экономика,Бизнес,Финансы,Дело,Биржа,Рынок,Акции,Прогнозы,Критика,Интервью,Рейтинги,Документы,Деньги,Власть,Автопилот,Тематические страницы,Первые лица,Деловые новости,Мировая практика,Культура,Спорт,Weekend,Астрологический прогноз,Погода мира,Курсы валют ЦБ РФ" />\r\n\r\n    <meta name="yandex-verification" content="50df68945a519dbd" />\r\n\r\n    \r\n<meta name="viewport" content="width=device-width, initial-scale=

In [ ]:
soup = BeautifulSoup(res.text)
soup

In [ ]:
# собираем все теги со ссылками на полные тексты новостей
refs = soup.find_all('a', class_='uho__link')
print(len(refs))
print(refs)

In [ ]:
# добираемся до ссылок
all_links = []

for ref in refs:
    all_links.append(ref.get('href'))

print(len(all_links))
print(all_links)

In [ ]:
# исключаем дубли
all_links = set(all_links)
print(len(all_links))

In [ ]:
# формируем полноценные ссылки
all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))
print(all_full_links)

In [ ]:
# объединим все в одну функцию
def get_all_links(query):
    url = 'https://www.kommersant.ru/search/results'
    params = {
        'search_query': query,
    }
    res = requests.get(URL, params)
    soup = BeautifulSoup(res.text)
    refs = soup.find_all('a', class_='uho__link')

    all_links = []
    for ref in refs:
        all_links.append(ref.get('href'))
    
    all_links = set(all_links)

    all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))

    return all_full_links

all_links = get_all_links('python')
print(all_links)

In [ ]:
# но мы же собрали только одну страницу? Хотим ВСЕ новости
def get_all_links(query, pages):
    url = 'https://www.kommersant.ru/search/results'
    links_list = []
    params = {
        'search_query': query
    }
    for i in range(1, pages+1):
        params['page'] = i
        res = requests.get(URL, params)
        time.sleep(0.33)

        refs = soup.find_all('a', class_='uho__link')
        all_links = []

        for ref in refs:
            all_links.append(ref.get('href'))
        
        all_links = set(all_links)

        all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))

        links_list += all_full_links
    return links_list

all_links = get_all_links('python', 3)
print(len(all_links))
print(all_links)

In [ ]:
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.33)
    date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
    print(date)
    title = soup.find('h1', class_='doc_header__name').text.strip()
    print(title)
    text = soup.find('div', class_='doc__body').text.strip()
    print(text)

In [ ]:
# запишем данные в датафрейм
kom_news = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
    title = soup.find('h1', class_='doc_header__name').text.strip()
    text = soup.find('div', class_='doc__body').text.strip().replace('\n', '')
    row = {'date': date, 'title': title, 'text': text}
    kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news

In [ ]:
# обернем в функцию 
def get_kom_news(links):
    kom_news = pd.DataFrame()
    for link in all_links:
        soup = BeautifulSoup(requests.get(link).text)
        time.sleep(0.3)
        date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
        title = soup.find('h1', class_='doc_header__name').text.strip()
        text = soup.find('div', class_='doc__body').text.strip().replace('\n', '')
        row = {'date': date, 'title': title, 'text': text}
        kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
    return kom_news.reset_index(drop=True)

In [ ]:
kom_news = get_kom_news(all_links)
display(kom_news)